In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Load the image
image_path = "docs/scan.jpeg"  # Replace with your image path
output_path = "docs/output_cleaned.jpg"  # Replace with desired output path
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Could not read image at {image_path}")


In [2]:
# Convert to HSV color space for better blue detection
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define range for blue color (adjust these values based on your image)
lower_blue = np.array([100, 50, 50])  # Lower bound for blue in HSV
upper_blue = np.array([130, 255, 255])  # Upper bound for blue in HSV
blue_mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

# Invert the blue mask to use it for removing blue ink
blue_mask_inv = cv2.bitwise_not(blue_mask)

# Create a mask for black content (low intensity in all channels)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
black_mask = cv2.threshold(gray_image, 110, 255, cv2.THRESH_BINARY_INV)[1]  # Adjust threshold (100) as needed

# Combine masks to keep black content and remove blue ink
combined_mask = cv2.bitwise_and(black_mask, blue_mask_inv)

# Apply the mask to the original image to retain only black content
clean_image = cv2.bitwise_and(image, image, mask=combined_mask)

# Convert to grayscale for enhancement
clean_gray = cv2.cvtColor(clean_image, cv2.COLOR_BGR2GRAY)

# Enhance contrast using CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced = clahe.apply(clean_gray)

# Denoise the image
denoised = cv2.fastNlMeansDenoising(enhanced, None, 10, 7, 21)

# Apply binary thresholding to get black text on white background
_, binary = cv2.threshold(denoised, 200, 255, cv2.THRESH_BINARY)

# Apply morphological operations with an adjustable kernel
kernel_size = 51  # Start here, can increase up to 51 or 101
kernel = np.ones((kernel_size, kernel_size), np.uint8)
cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)
cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel, iterations=1)

# Create the final output image (white background with black text)
output_image = np.ones_like(image) * 255
output_image[combined_mask > 0] = 0  # Set black regions
output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for simplicity


In [3]:
# Save the result
cv2.imwrite(output_path, output_image)
print(f"Image saved to {output_path}")

Image saved to docs/output_cleaned.jpg
